# Visualize the Raw Visual Attentions Behavior

In [1]:
import pandas as pd
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
import json
import modules.GenSnippetsLib as snippets
import scipy.ndimage
from pygments.lexers import PythonLexer
from tqdm.notebook import tqdm
import matplotlib.patches as patches
import PIL
import os
import PIL.Image
from skimage.draw import ellipse
import numpy as np
plt.rcParams['savefig.facecolor']='white'

## Load the data

In [2]:
def click_data_to_list(click_data_string):
    click_data_list = []
    for click in click_data_string.split(" "):
        click = click.split("-")
        # extract the x and y coordinates from a string via regex
        x = int(re.search(r'\d+', click[0])[0])
        y = int(re.search(r'\d+', click[1])[0])
        click_data_list.append((x, y))
    return click_data_list#

def time_data_to_list(time_data_string):
    time_data_list = []
    for time in time_data_string.split(" "):
        time_data_list.append(float(time) / 1000)
    return time_data_list

In [3]:
df = pd.read_csv('./data/preprocessed_experiment_data.csv')
df = df[df["Outlier"] == False]
df = df.drop(columns=["Outlier"])

df["ClickData"] = df["ClickData"].apply(lambda x: click_data_to_list(x))
df["TimeData"] = df["TimeData"].apply(lambda x: time_data_to_list(x))
df["NumberOfClicks"] = df["ClickData"].apply(lambda x: len(x))
df["AverageTimeOfClicks"] = (df["ResponseTime"] / 1000) / df["NumberOfClicks"]

## Create Heatmaps

In [4]:
def get_ellipse_mask(center, x_radius, y_radius, dimension, std_value = 1.0):
    """
    Returns a mask of an ellipse with the given center, x-radius, and y-radius.
    """
    x_dim = max(center[0]+2*x_radius+1, dimension[0])
    y_dim = max(center[1]+2*y_radius+1, dimension[1])
    mask = np.zeros((y_dim, x_dim), dtype=np.float32)
    rr, cc = ellipse(center[1], center[0], y_radius, x_radius)
    mask[rr, cc] = std_value
    mask = mask[:dimension[1], :dimension[0]]
    return mask

def heatmap_helper(image, heat_array, grad_radius, treshhold=0.2):
    cmap = plt.cm.get_cmap('jet')

    heat_array = heat_array.copy()
    heat_array = scipy.ndimage.gaussian_filter(heat_array, sigma=grad_radius)
    max_value = np.max(heat_array)
    heat_array = heat_array / max_value
    key_out_mask = heat_array > treshhold
    heat_normalized = heat_array.copy()
    heat_array = heat_array*255
    heat_array = heat_array.astype(np.uint8)
    heat_array = cmap(heat_array)
    heat_array = heat_array*255

    hmap = np.array(image)
    hmap[key_out_mask] = 0.5*hmap[key_out_mask] + 0.5*heat_array[key_out_mask]
    return hmap, heat_normalized, max_value

In [5]:
settings = json.load(open("./data/settings.json"))
x_radius = settings["radius_x"]+settings["grad_radius"]//2
y_radius = settings["radius_y"]+settings["grad_radius"]//2

In [6]:
# check if folder exists
if not os.path.exists("./results/average_heatmaps"):
    os.makedirs("./results/average_heatmaps")

for group_name, df_group in tqdm(df.groupby(["Algorithm", "ProgrammingStyle"])):
    image = "./data/images/" + group_name[0] + "_" + group_name[1].replace("-", "_") + ".png"
    image = PIL.Image.open(image)
    dimension = image.size
    average_heat = np.zeros((dimension[1], dimension[0]))
    for idx, row in df_group.iterrows():
        clicks = row["ClickData"]
        duration_per_click = row["TimeData"]
        duration_tmp = [duration_per_click[0]]
        for duration_idx, value in enumerate(duration_per_click):
            if duration_idx == 0:
                continue
            duration_tmp.append(value-duration_per_click[duration_idx-1])
        duration_per_click = duration_tmp

        example = np.zeros((dimension[1], dimension[0]))
        for click, duration in zip(clicks, duration_per_click):
            elliptical_mask = get_ellipse_mask(click, x_radius, y_radius, dimension)
            example += (elliptical_mask*duration)
        _hmap, normalized_heat, _max_value = heatmap_helper(image, example, settings["grad_radius"]//2)
        average_heat += normalized_heat

    image = np.array(image)

    average_heat = scipy.ndimage.gaussian_filter(average_heat, sigma=4)
    average_heat = average_heat / len(df_group)
    mask = average_heat >= 0.00

    cmap = plt.cm.get_cmap('jet')
    average_heat = average_heat*255
    average_heat = average_heat.astype(np.uint8)
    average_heat = cmap(average_heat)
    average_heat = average_heat*255

    image[mask] = 0.5*image[mask] + 0.5*average_heat[mask]

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    fig.set_size_inches(dimension[0]/100 * 1.1, dimension[1]/100)
    sm = plt.cm.ScalarMappable(cmap=plt.cm.get_cmap("jet"))
    plt.axis('off')
    plt.tight_layout()
    ax.imshow(image, aspect='auto')
    plt.colorbar(sm, fraction=0.046, pad=0.04)
    plt.savefig("./results/average_heatmaps/" + group_name[0] + "_" + group_name[1].replace("-", "_") + ".png", bbox_inches='tight' , pad_inches=0)
    plt.close('all')

  0%|          | 0/44 [00:00<?, ?it/s]

## Create Perceptions Maps

In [7]:
# check if folder exists
if not os.path.exists("./results/average_persuasion_maps"):
    os.makedirs("./results/average_persuasion_maps")

for group_name, df_group in tqdm(df.groupby(["Algorithm", "ProgrammingStyle"])):
    image = "./data/images/" + group_name[0] + "_" + group_name[1].replace("-", "_") + ".png"
    image = PIL.Image.open(image)
    dimension = image.size
    average_heat = np.zeros((dimension[1], dimension[0]))
    for idx, row in df_group.iterrows():
        clicks = row["ClickData"]
        duration_per_click = row["TimeData"]
        duration_tmp = [duration_per_click[0]]
        for duration_idx, value in enumerate(duration_per_click):
            if duration_idx == 0:
                continue
            duration_tmp.append(value-duration_per_click[duration_idx-1])
        duration_per_click = duration_tmp

        example = np.zeros((dimension[1], dimension[0]))
        for click, duration in zip(clicks, duration_per_click):
            elliptical_mask = get_ellipse_mask(click, x_radius, y_radius, dimension)
            example += (elliptical_mask*duration)
        _hmap, normalized_heat, _max_value = heatmap_helper(image, example, settings["grad_radius"]//2)
        average_heat += normalized_heat

    image = np.array(image)

    average_heat = scipy.ndimage.gaussian_filter(average_heat, sigma=4)
    average_heat = average_heat / len(df_group)
    average_heat = 1.0 - average_heat

    cmap = plt.cm.get_cmap('Greys')
    average_heat = average_heat*255
    average_heat = average_heat.astype(np.uint8)
    average_heat = cmap(average_heat)
    average_heat = average_heat
    image_1 = image.copy()
    image_1 = image_1/255.0


    image = average_heat*image_1*0.95 + image_1*0.05

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    fig.set_size_inches(dimension[0]/100, dimension[1]/100)
    plt.axis('off')
    plt.tight_layout()
    ax.imshow(image, aspect='auto')
    plt.savefig("./results/average_persuasion_maps/" + group_name[0] + "_" + group_name[1].replace("-", "_") + ".png", bbox_inches='tight' , pad_inches=0)
    plt.close('all')

  0%|          | 0/44 [00:00<?, ?it/s]

## Create AOI Visualizations

In [8]:
# prepare color mapping for aois
algorithms = df[["Algorithm"]].sort_values(by="Algorithm")["Algorithm"].unique()
programming_styles = df[["ProgrammingStyle"]].sort_values(by="ProgrammingStyle")["ProgrammingStyle"].unique()
algorithms = [x for x in algorithms]
programming_styles = [x.replace("-", "_") for x in programming_styles]

color_palette = sns.color_palette("hls", 16)
color_palette_mapping = {
    "Method" : color_palette[0],
    "MethodDeclaration" : color_palette[1],
    "MethodBody" : color_palette[2],
    "Helper" : color_palette[3],
    "HelperDeclaration" : color_palette[4],
    "HelperBody" : color_palette[5],
    "Iteration" : color_palette[6],
    "IterationHead" : color_palette[7],
    "IterationBody" : color_palette[8],
    "RecursiveCall" : color_palette[9],
    "RecursiveBaseCase" : color_palette[10],
    "HigherOrder" : color_palette[11],
    "ListComprehension" : color_palette[12],
    "InputPreparation" : color_palette[13],
    "StartOfAlgorithm" : color_palette[14],
    "Class" : color_palette[15],
}
aoi_colors = color_palette
aoi_numbers = {name : idx for idx, (name, _value) in enumerate(color_palette_mapping.items())}
aoi_names = [name for name, _value in color_palette_mapping.items()]

def aoi_idx_to_convoluted_idx(aoi_idx):
    if aoi_idx < 5:
        return aoi_idx
    if 6 <= aoi_idx <= 12:
        return 6
    if 13 <= aoi_idx:
        return aoi_idx-6

def aoi_name_to_convoluted_name(aoi_name):
    if aoi_name in ["Iteration", "IterationHead", "IterationBody", "RecursiveCall", "RecursiveBaseCase", "HigherOrder", "ListComprehension"]:
        return "StyleSpecific"
    return aoi_name

color_palette = sns.color_palette("hls", 10)
color_palette_mapping_convoluted = {
    "Method" : color_palette[0],
    "MethodDeclaration" : color_palette[1],
    "MethodBody" : color_palette[2],
    "Helper" : color_palette[3],
    "HelperDeclaration" : color_palette[4],
    "HelperBody" : color_palette[5],
    "StyleSpecific" : color_palette[6],
    "InputPreparation" : color_palette[7],
    "StartOfAlgorithm" : color_palette[8],
    "Class" : color_palette[9],
}
aoi_colors_convoluted = color_palette
aoi_numbers_convoluted = {name : idx for idx, (name, _value) in enumerate(color_palette_mapping_convoluted.items())}
aoi_names_convoluted = [name for name, _value in color_palette_mapping_convoluted.items()]

In [9]:
# create bounding boxes and visualize them
def from_canvas(fig):
    lst = list(fig.canvas.get_width_height())
    lst.append(3)
    return PIL.Image.fromarray(np.frombuffer(fig.canvas.tostring_rgb(),dtype=np.uint8).reshape(lst))

df_bounding_box = pd.DataFrame(columns=["Algorithm", "ProgrammingStyle", "BoundingBoxes", "ImageDimension"])
image_path = "./results/aois/"
font_path="\\data\\fonts\\ttf\\"
offset = 3

# check if the image_path exists
if not os.path.exists(image_path):
    os.makedirs(image_path)

for algorithm in tqdm(algorithms):
    for programming_style in programming_styles:
        # load the image and generate bb code info
        file_name = './data/CodeSnippets/Generators/' + algorithm + '_' + programming_style + '.json'
        try:
            image, result = snippets.create_image(file_name, font_path=font_path, lexer=PythonLexer)
            y, x = image.size[::-1]
            dimension = (x, y)
        except Exception as e:
            print(file_name + ":" + str(e))
            continue
        # extract the areas of interest in the image
        areas_of_interest = set()
        for letter in result:
            for aoi in letter["AOI"]:
                areas_of_interest.add(aoi)

        # create the bounding boxes
        disjoint_areas_of_interest = []
        for aoi in areas_of_interest:
            upper_left_x = 0
            upper_left_y = 0
            lower_right_x = 0
            lower_right_y = 0
            has_aoi = False
            for letter in result:
                if letter["BoundingBox"][0] == 0 and letter["BoundingBox"][1] == 0 and letter["BoundingBox"][2] == 0 and letter["BoundingBox"][3] == 0:
                    continue

                if aoi in letter["AOI"]:
                    if not has_aoi:
                        upper_left_x = letter["BoundingBox"][0] - offset
                        upper_left_y = letter["BoundingBox"][1] - offset
                        lower_right_x = letter["BoundingBox"][2] + offset
                        lower_right_y = letter["BoundingBox"][3] + offset
                        has_aoi = True
                    else:
                        upper_left_x = min(upper_left_x, letter["BoundingBox"][0] - offset)
                        upper_left_y = min(upper_left_y, letter["BoundingBox"][1] - offset)
                        lower_right_x = max(lower_right_x, letter["BoundingBox"][2] + offset)
                        lower_right_y = max(lower_right_y, letter["BoundingBox"][3] + offset)
                else:
                    if has_aoi:
                        disjoint_areas_of_interest.append((aoi, [upper_left_x, upper_left_y, lower_right_x, lower_right_y]))
                    has_aoi = False
                    upper_left_x = 0
                    upper_left_y = 0
                    lower_right_x = 0
                    lower_right_y = 0

            if has_aoi:
                disjoint_areas_of_interest.append((aoi, [upper_left_x, upper_left_y, lower_right_x, lower_right_y]))

        # create the bounding boxes without None aoi
        disjoint_areas_of_interest = [area for area in disjoint_areas_of_interest if area[0] != "None"]
        df_bounding_box = df_bounding_box.append(pd.DataFrame([[algorithm, programming_style.replace("_", "-"), disjoint_areas_of_interest, dimension]], columns=df_bounding_box.columns))

        # map a color to each aoi
        disjoint_areas_of_interest_visual = []
        for aoi, bounding_box in disjoint_areas_of_interest:
            h, s, l = color_palette_mapping[aoi]
            color = (h, s, l, 0.2)
            disjoint_areas_of_interest_visual.append((aoi, bounding_box, color))

        # create the image with the bounding boxes
        fig, ax = plt.subplots(1, 1)
        ax.imshow(image)

        for (aoi, bounding_box, color) in disjoint_areas_of_interest_visual:
            rect = patches.Rectangle((bounding_box[0], bounding_box[1]), bounding_box[2] - bounding_box[0],
                                     bounding_box[3] - bounding_box[1], linewidth=1, edgecolor=color, facecolor=color)
            ax.add_patch(rect)

        plt.tight_layout()
        plt.axis('off')
        plt.savefig(image_path + "aoi_" + algorithm + '_' + programming_style + '.png', bbox_inches='tight', transparent=True, pad_inches=0)
        plt.close('all')

  0%|          | 0/11 [00:00<?, ?it/s]